## Preparing COVID-19 Dataset for Visualization

### Data Cleaning

In [1]:
import pandas as pd
import os

# import data
data_path = "./covid19-data"

d1 = pd.read_csv(os.path.join(data_path,
                 "DOH COVID Data Drop_ 20230311 - 04 Case Information_batch_0.csv"),
                 low_memory=False)
d2 = pd.read_csv(os.path.join(data_path,
                 "DOH COVID Data Drop_ 20230311 - 04 Case Information_batch_1.csv"),
                 low_memory=False)
d3 = pd.read_csv(os.path.join(data_path,
                 "DOH COVID Data Drop_ 20230311 - 04 Case Information_batch_2.csv"),
                 low_memory=False)
d4 = pd.read_csv(os.path.join(data_path,
                 "DOH COVID Data Drop_ 20230311 - 04 Case Information_batch_3.csv"),
                 low_memory=False)
d5 = pd.read_csv(os.path.join(data_path,
                 "DOH COVID Data Drop_ 20230311 - 04 Case Information_batch_4.csv"),
                 low_memory=False)
data = pd.concat([d1, d2, d3, d4, d5])

data.describe()

,Age
count,4.066352e+06
mean,3.784972e+01
std,1.800668e+01
min,-1.000000e+00
25%,2.600000e+01
50%,3.500000e+01
75%,5.000000e+01
max,1.070000e+02


Observe that the data from each respective csv files have been combined in the `data` variable. We now check the status of missing entries from each column of the data.

In [7]:
# check missing numbers
import missingno as msno

# msno.matrix(data)

Observe that the _DateSpecimen_, _DateResultReleased_, _DateDied_ _DateRecovered_, _DateOnset_, and _Pregnanttab_ columns have a lot of missing entries. Meanwhile, some missing entries are observed for the _Age_, _AgeGroup_, _RemovalType_, _Admitted_, _ProvRes_, _CityMunRes_, _CityMuniPSGC_, _BarangayRes_, and _BarangayPSGC_ columns.

From this, we remove the colums having a lot of missing entries.

In [ ]:
data = data.drop(['DateSpecimen', 'DateResultRelease', 'DateDied', 'DateRecover', 'DateOnset', 'Pregnanttab'], axis=1)

# msno.matrix(data)

We also remove some columns that will not be included in our analysis.

In [ ]:
data = data.drop(['RemovalType', 'Admitted', 'RegionRes', 'CityMuniPSGC', 'BarangayRes', 'BarangayPSGC',
                    'HealthStatus', 'Quarantined', 'ValidationStatus'], axis=1)

# msno.matrix(data)

We now remove rows with missing values in the remaining columns.

In [28]:
data_cleaned = data.copy().dropna()
print(f'Original count: \n {data.count()} \n')
print(f'New count: \n {data_cleaned.count()}')

Original count: 
 CaseCode       4077757
Age            4066352
AgeGroup       4066352
Sex            4077756
DateRepConf    4077757
ProvRes        4020420
CityMunRes     3987842
dtype: int64 

New count: 
 CaseCode       3977973
Age            3977973
AgeGroup       3977973
Sex            3977973
DateRepConf    3977973
ProvRes        3977973
CityMunRes     3977973
dtype: int64


Let's look at the description of the remaining columns to give us a context in our analysis.

In [ ]:
metadata = pd.read_csv(os.path.join(data_path, 
                                    "DOH COVID Data Drop_ 20230422 - 03 Metadata - Fields.csv"))

In [ ]:
metadata[['Field', 'Description', 'Type']].iloc[[0, 1, 2, 3, 6, 12, 13]]

,Field,Description,Type
0,CaseCode,Random code assigned for labelling cases; does...,Text
1,Age,Age,Number
2,AgeGroup,Five-year age group,Text
3,Sex,Sex,Text
6,DateRepConf,Date publicly announced as confirmed case,Date
12,ProvRes,Province of residence,Text
13,CityMunRes,City of residence,Text


In [ ]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3977973 entries, 0 to 77756
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   CaseCode     object 
 1   Age          float64
 2   AgeGroup     object 
 3   Sex          object 
 4   DateRepConf  object 
 5   ProvRes      object 
 6   CityMunRes   object 
dtypes: float64(1), object(6)
memory usage: 242.8+ MB


### Data Structuring

Let's now structure our data to our preferred format:
1. Convert the date column to the correct format.
2. Use _F_ or _M_ for the sex column.
3. Filter values from 2020 to 2022 only.
4. Change the locations entries to their corresponding coordinates.

In [9]:
data_cleaned['DateRepConf'] = pd.to_datetime(data_cleaned['DateRepConf'])

In [10]:
data_cleaned.loc[data_cleaned['Sex'] == 'FEMALE', 'Sex'] = 'F'
data_cleaned.loc[data_cleaned['Sex'] == 'MALE', 'Sex'] = 'M'

In [11]:
data_cleaned = data_cleaned[data_cleaned['DateRepConf'].dt.year.isin([2020,2021,2022])]

In [82]:
# convert the city and province locations into latitude, longitude format

from tqdm import tqdm
from geopy.geocoders import Nominatim
loc = Nominatim(user_agent='AM255')


# convert unique provinces and city/municipality

loc_cols = ['ProvRes', 'CityMunRes']  # location columns
loc_latlong = {col: [] for col in loc_cols}

In [83]:
# getting coordinates of provinces
loc_latlong['ProvRes'] = []
for idx, location in tqdm(enumerate(data_cleaned['ProvRes'].unique())):
    get_loc = loc.geocode(location + ' Philippines', timeout=None)

    if get_loc is None:
        loc_latlong['ProvRes'].append([location, None])
    else:
        loc_latlong['ProvRes'].append([location, str(get_loc.latitude) + ', ' + str(get_loc.longitude)])

84it [01:31,  1.09s/it]


In [ ]:
# getting coordinates of city/municipalities
# loc_latlong['CityMunRes'] = []
for idx, location in tqdm(enumerate(data_cleaned['CityMunRes'].unique())):
    get_loc = loc.geocode(location + ' Philippines', timeout=None)

    if get_loc is None:
        loc_latlong['CityMunRes'].append([location, None])
    else:
        loc_latlong['CityMunRes'].append([location, str(get_loc.latitude) + ', ' + str(get_loc.longitude)])

1436it [24:06,  1.01s/it]


In [208]:
# manually input coordinates of locations not detected by geopy

# prov_idx = [idx for idx in range(len(loc_latlong['ProvRes'])) if loc_latlong['ProvRes'][idx, 1] is None]
citmun_idx = [idx for idx in range(len(loc_latlong['CityMunRes'])) if loc_latlong['CityMunRes'][idx, 1] is None]

# loc_latlong['ProvRes'][prov_idx, 0]
# loc_latlong['ProvRes'][prov_idx, 1] = ['7.2047, 124.2310', '6.7029, 121.9690']

loc_latlong['CityMunRes'][citmun_idx, 1] = [
    '15.067632, 120.648920', '9.7638, 118.7473', '17.6132, 121.7270', '14.8527, 120.8160', '13.3941, 121.8790',
    '13.3771, 121.1646', '15.4755, 120.5963', '16.4804, 121.1481', '7.4471, 125.8094', '6.9522, 126.2173',
    '14.6799, 120.5421', '6.7388, 125.3549', '16.1505, 119.9856', '7.9943, 123.8746', '13.1028, 123.6959',
    '7.0106, 125.0911', '6.4974, 124.8472', '16.9166, 121.7879', '13.2407, 123.5380', '11.8240, 124.8403',
    '6.4118, 125.6132', '6.8531, 124.3976', '6.6615, 122.1065', '17.4117, 121.4384', '8.1479, 125.1321',
    '16.8083, 121.1939', '7.8345, 124.3477', '7.7821, 124.4642', '7.92, 124.2', '10.8412, 123.4991',
    '12.9700, 124.0030', '7.8735, 124.1254', '10.1654, 124.8403', '17.5705, 120.3873', '10.0988, 122.8710',
    '10.2704, 123.0758', '7.3001, 125.9370', '17.1874, 120.4467', '7.3362, 126.1328', '8.3924, 123.4271',
    '9.4444, 123.1801', '10.1206, 123.2717', '8.3803, 123.0564', '18.0172, 121.1842', '13.7367, 123.7406',
    '13.5849, 124.2066', '15.3634, 121.2168', '9.7198, 122.4920', '13.2978, 122.5599', '7.6038, 125.9632',
    '12.3574, 123.5504', '17.5882, 120.6315', '13.2682, 120.6205', '5.1042, 119.8121', '6.1042, 125.2879',
    '17.4930, 121.6055', '8.0042, 123.4608', '11.6080, 125.4329', '8.5183, 123.2328', '9.1012, 126.1589',
    '17.1503, 120.5481', '7.7431, 120.4583', '8.9622, 125.2039', '9.2322, 124.7078', '8.5618, 124.5246',
    '17.1031, 120.6890', '10.0192, 126.0387', '16.4199, 121.5142', '10.121, 124.553', '7.7488, 124.1198',
    '14.1189, 122.8710', '10.1039, 125.5767', '17.1188, 120.6186', '5.9564, 121.1825'
]

In [ ]:
df_prov = pd.DataFrame(loc_latlong['ProvRes'], columns=['ProvRes', 'ProvCoord'])
df_citmun = pd.DataFrame(loc_latlong['CityMunRes'], columns=['CityMunRes', 'CityMunCoord'])

df_prov.to_csv(os.path.join(data_path, 'coords_prov.csv'), index=False)
df_citmun.to_csv(os.path.join(data_path, 'coords_citmun.csv'), index=False)

In [219]:
# join the coordinates to the dataset
data_cleaned = pd.merge(data_cleaned, df_prov, on='ProvRes', how='left')
data_cleaned = pd.merge(data_cleaned, df_citmun, on='CityMunRes', how='left')
data_cleaned

,CaseCode,Age,AgeGroup,Sex,DateRepConf,ProvRes,CityMunRes,ProvCoord,CityMunCoord
0,C404174,38.0,35 to 39,FEMALE,2020-01-30,NEGROS ORIENTAL,DUMAGUETE CITY (CAPITAL),"9.75, 123.0","14.651068321895105, 121.03147238313927"
1,C462688,44.0,40 to 44,MALE,2020-02-03,NEGROS ORIENTAL,DUMAGUETE CITY (CAPITAL),"9.75, 123.0","14.651068321895105, 121.03147238313927"
2,C387710,60.0,60 to 64,FEMALE,2020-02-05,BOHOL,PANGLAO,"9.833333, 124.1615579","9.5791851, 123.7452642"
3,C498051,63.0,60 to 64,MALE,2020-03-06,RIZAL,CAINTA,"14.65, 121.25","14.5780161, 121.1163072"
4,C377460,49.0,45 to 49,MALE,2020-03-06,BATANGAS,SANTO TOMAS,"13.9146826, 121.0867566","14.1078443, 121.1453304"
...,...,...,...,...,...,...,...,...,...
3977968,C52803057,29.0,25 to 29,MALE,2023-03-11,NCR,CITY OF MANILA,"14.5736108, 121.0329706","14.5904492, 120.9803621"
3977969,C21585070,5.0,5 to 9,MALE,2023-03-11,NCR,CITY OF PASIG,"14.5736108, 121.0329706","14.5605166, 121.0764343"
3977970,C8641682,28.0,25 to 29,MALE,2023-03-11,NCR,CITY OF PASIG,"14.5736108, 121.0329706","14.5605166, 121.0764343"
3977971,C66948100,28.0,25 to 29,FEMALE,2023-03-11,PALAWAN,SOFRONIO ESPAÑOLA,"11.0798978, 120.93734215474105","8.9636169, 118.0004439"


### Separating and Exporting Datasets

In [8]:
data_cleaned.to_csv(os.path.join(data_path, 'covid-data-cleaned.csv'), index=False)

In [2]:
import pandas as pd
import os

# import data
data_path = "./covid19-data"
data_cleaned = pd.read_csv(os.path.join(data_path, 'covid-data-cleaned.csv'))

In [ ]:
data_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3977973 entries, 0 to 3977972
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   CaseCode      object 
 1   Age           float64
 2   AgeGroup      object 
 3   Sex           object 
 4   DateRepConf   object 
 5   ProvRes       object 
 6   CityMunRes    object 
 7   ProvCoord     object 
 8   CityMunCoord  object 
dtypes: float64(1), object(8)
memory usage: 273.1+ MB


In [7]:
data_cleaned['DateRepConf'] = pd.to_datetime(data_cleaned['DateRepConf'])

In [10]:
data_cleaned = data_cleaned[data_cleaned['DateRepConf'].dt.year.isin([2020,2021,2022])]

In [24]:
data_cleaned['Age'].unique()

array([ 38.,  44.,  60.,  63.,  49.,  58.,  39.,  86.,  46.,  70.,  31.,
        68.,  41.,  52.,  48.,  51.,  57.,  25.,  59.,  34.,  82.,  29.,
        64.,  43.,  73.,  67.,  75.,  76.,  65.,  53.,  66.,  28.,  54.,
        50.,  72.,  79.,  26.,  40.,  33.,  45.,  35.,  32.,  23.,  69.,
        77.,  55.,  42.,  27.,  71.,  88.,  61.,  30.,  47.,  36.,  13.,
        74.,  56.,  78.,  37.,  21.,  62.,  24.,  89.,  20.,  80.,  84.,
        19.,  83.,  81.,  87.,  93.,  96.,  22.,   5.,  85.,   1.,  17.,
        18.,  98.,   2.,  90.,  92.,  91.,   3.,   8.,  15.,   0.,   7.,
        11.,  16.,   9.,  12.,   4.,  14.,  10.,  94.,   6.,  97.,  95.,
        99.,  -1., 101., 103., 100., 107., 102., 105., 104., 106.])

In [ ]:
data_demog = data_cleaned[['CaseCode', 'Age', 'Sex']]  # age and sex
data_prov = data_cleaned[['CaseCode', 'ProvRes']]
data_citmun = data_cleaned[['CaseCode', 'CityMunRes']]
data_date = data_cleaned[['CaseCode', 'DateRepConf']]

In [6]:
# export into csv files
data_demog.to_csv(os.path.join(data_path, 'covid-demog.csv'), index=False)
data_prov.to_csv(os.path.join(data_path, 'covid-prov.csv'), index=False)
data_citmun.to_csv(os.path.join(data_path, 'covid-citmun.csv'), index=False)
data_date.to_csv(os.path.join(data_path, 'covid-date.csv'), index=False)

In [15]:
data_loc = data_cleaned[['CaseCode', 'ProvCoord', 'CityMunCoord']]
data_loc.to_csv(os.path.join(data_path, 'covid-loc.csv'), index=False)